In [18]:
import os

def find_extlist(path_dataset, extension='.txt'):
    file_list = []
    for (path, dir, files) in os.walk(path_dataset):
        for filename in files:
            ext = os.path.splitext(filename)[-1]
            if ext == extension:
                file_list.append(path + "/" + filename)

    return file_list

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

from scipy import io
def mat2txt(mat_path, scene_path):
    mat_file = io.loadmat(mat_path)
    file_name = mat_path.split('/')[-1]
    file_name = os.path.splitext(file_name)[0] + '.txt'
    f = open(scene_path + '/' + file_name, 'w')
    for point in mat_file['image_info'][0][0][0][0][0]:
        contents = str(point[0]) + ' ' + str(point[1]) + '\n'
        f.writelines(contents)
    f.close()

import json
def json2txt(json_path, save_path):
    with open(json_path, "r") as file:
        json_file = json.load(file)
    txt_name = json_path.split('/')[-1]
    txt_name = os.path.splitext(txt_name)[0] + '.txt'
    f = open(save_path + '/' + txt_name, 'w')
    points = json_file['points']
    for point in points:
        contents = str(point[0]) + ' '+ str(point[1]) + '\n'
        f.writelines(contents)
    f.close()   
     
def makelist(source_path):
    img_path_list = find_extlist(source_path, '.jpg')
    spl = '/'.join([source_path.split('/')[-3], source_path.split('/')[-2]]) + '/'
    file_path = source_path + '/../' + source_path.split('/')[-1] + '.list'
    file = open(file_path, 'w')
    for img_path in img_path_list:
        img_path = img_path.split(spl)[-1]
        mat_path = os.path.splitext(img_path)[0] + '.txt'
        contents = img_path + ' ' + mat_path + '\n'
        file.writelines(contents)
    file.close()



In [ ]:
path = '/usr/src/app/CrowdCounting-P2PNet/NWPUCrowd'
txt_path = path + '/annotation_txt'
createFolder(txt_path)
json_list = find_extlist(path, '.json')

for file in json_list:
    json2txt(file, txt_path)

In [ ]:
import cv2
file = open('/usr/src/app/CrowdCounting-P2PNet/NWPUCrowd/train.txt', 'r')
lines = file.read().split('\n')
file.close()

for idx, line in enumerate(lines):
    img_file = '/usr/src/app/CrowdCounting-P2PNet/NWPUCrowd/images/' + line.split(' ')[0] + '.jpg'
    if line.split(' ')[0] == '':
        continue
    txt_file = '/usr/src/app/CrowdCounting-P2PNet/NWPUCrowd/annotation_txt/' + line.split(' ')[0] + '.txt'
    img = cv2.imread(img_file)
    h, w, c =img.shape
    ratio_h = 1080. / h
    ratio_w = 1920. / w
    r = min(ratio_w, ratio_h)
    new_shape_h = int(h * r)
    new_shape_w = int(w * r)
    
    if r < 1.:
        resize_img = cv2.resize(img, (new_shape_w, new_shape_h), interpolation=cv2.INTER_AREA)
    else:
        resize_img = cv2.resize(img, (new_shape_w, new_shape_h), interpolation=cv2.INTER_LINEAR)
    os.mkdir('/usr/src/app/CrowdCounting-P2PNet/datasets/ver4/train/scene'+str(idx+1))
    cv2.imwrite('/usr/src/app/CrowdCounting-P2PNet/datasets/ver4/train/scene' + str(idx+1) + '/IMG_' + str(idx+1) + '.jpg', resize_img)
    file = open(txt_file, 'r')
    txt_lines = file.read().split('\n')
    file.close()
    contents = ''
    for txt_line in txt_lines:
        if txt_line == '':
            break
        try:
            txt_line = txt_line.split(' ')
            contents += str(float(txt_line[0]) * r) + ' ' + str(float(txt_line[1]) * r) + '\n'
        except Exception as e:
            print(txt_line)
    file = open('/usr/src/app/CrowdCounting-P2PNet/datasets/ver4/train/scene' + str(idx+1) + '/GT_IMG_' + str(idx+1) + '.txt', 'w')
    file.writelines(contents)
    file.close()
makelist('/usr/src/app/CrowdCounting-P2PNet/datasets/ver4/train')

In [ ]:
file = open('/usr/src/app/CrowdCounting-P2PNet/NWPUCrowd/val.txt', 'r')
lines = file.read().split('\n')
file.close()

for idx, line in enumerate(lines):
    if line.split(' ')[0] == '':
        continue
    img_file = '/usr/src/app/CrowdCounting-P2PNet/NWPUCrowd/images/' + line.split(' ')[0] + '.jpg'
    txt_file = '/usr/src/app/CrowdCounting-P2PNet/NWPUCrowd/annotation_txt/' + line.split(' ')[0] + '.txt'
    img = cv2.imread(img_file)
    h, w, c = img.shape

    ratio_h = 1080. / h
    ratio_w = 1920. / w
    r = min(ratio_w, ratio_h)
    new_shape_h = int(h * r)
    new_shape_w = int(w * r)
    
    if r < 1.:
        resize_img = cv2.resize(img, (new_shape_w, new_shape_h), interpolation=cv2.INTER_AREA)
    else:
        resize_img = cv2.resize(img, (new_shape_w, new_shape_h), interpolation=cv2.INTER_LINEAR)
    os.mkdir('/usr/src/app/CrowdCounting-P2PNet/datasets/ver4/test/scene'+str(idx+1))
    cv2.imwrite('/usr/src/app/CrowdCounting-P2PNet/datasets/ver4/test/scene' + str(idx+1) + '/IMG_' + str(idx+1) + '.jpg', resize_img)
    file = open(txt_file, 'r')
    txt_lines = file.read().split('\n')
    file.close()
    contents = ''
    for txt_line in txt_lines:
        if txt_line == '':
            break
        try:
            txt_line = txt_line.split(' ')
            contents += str(float(txt_line[0]) * r) + ' ' + str(float(txt_line[1]) * r) + '\n'
        except Exception as e:
            print(txt_line)

    file = open('/usr/src/app/CrowdCounting-P2PNet/datasets/ver4/test/scene' + str(idx+1) + '/GT_IMG_' + str(idx+1) + '.txt', 'w')
    file.writelines(contents)
    file.close()
makelist('/usr/src/app/CrowdCounting-P2PNet/datasets/ver4/test')

In [ ]:
makelist('/usr/src/app/CrowdCounting-P2PNet/datasets/ver1/train')
makelist('/usr/src/app/CrowdCounting-P2PNet/datasets/ver1/test')

In [ ]:
import numpy as np
import torch
def gen_discrete_map(im_height, im_width, points):
    """
        func: generate the discrete map.
        points: [num_gt, 2], for each row: [width, height]
        """
    discrete_map = np.zeros([im_height, im_width], dtype=np.float32)
    h, w = discrete_map.shape[:2]
    num_gt = points.shape[0]
    if num_gt == 0:
        return discrete_map
    
    # fast create discrete map
    points_np = np.array(points).round().astype(int)
    p_h = np.minimum(points_np[:, 1], np.array([h-1]*num_gt).astype(int))
    p_w = np.minimum(points_np[:, 0], np.array([w-1]*num_gt).astype(int))
    p_index = torch.from_numpy(p_h* im_width + p_w)
    discrete_map = torch.zeros(im_width * im_height).scatter_add_(0, index=p_index, src=torch.ones(im_width*im_height)).view(im_height, im_width).numpy()

    ''' slow method
    for p in points:
        p = np.round(p).astype(int)
        p[0], p[1] = min(h - 1, p[1]), min(w - 1, p[0])
        discrete_map[p[0], p[1]] += 1
    '''
    assert np.sum(discrete_map) == num_gt
    
    return discrete_map

In [ ]:
import json
import random

def json2yolotxt(json_path, save_path):
    with open(json_path, "r") as file:
        json_file = json.load(file)
    txt_name = json_path.split('/')[-1]
    txt_name = os.path.splitext(txt_name)[0] + '.txt'
    f = open(save_path + '/' + txt_name, 'w')
    boxes = json_file["boxes"]
    for box in boxes:
        # x1, y1, x2, y2
        contents = str(box[0]) + ' '+ str(box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + '\n'
        f.writelines(contents)
    f.close()   

path = '/usr/src/app/CrowdCounting-P2PNet/NWPUCrowd'
txt_path = '/usr/src/app/CrowdCounting-P2PNet/NWPUCrowd/annotation_box_txt'
createFolder(txt_path)
json_list = find_extlist(path, '.json')

for file in json_list:
    json2yolotxt(file, txt_path)

In [20]:
import cv2
import random
import os
txt_list = find_extlist('/usr/src/app/CrowdCounting-P2PNet/NWPUCrowd/annotation_box_txt', '.txt')
save_img_path = '/usr/src/app/CrowdCounting-P2PNet/NWPUCrowd/images'
save_txt_path = '/usr/src/app/CrowdCounting-P2PNet/NWPUCrowd/labels'

seq = list(range(len(txt_list)))
random.shuffle(seq)

for idx, value in enumerate(seq):
    if idx > (len(seq) * 0.8):
        train = False
    else:
        train = True

    img_file = '/usr/src/app/CrowdCounting-P2PNet/NWPUCrowd/images/' + os.path.splitext(txt_list[value].split('/')[-1])[0] + '.jpg'
    img = cv2.imread(img_file)
    h, w, c = img.shape

    ratio_h = 1280. / h
    ratio_w = 1280. / w
    r = min(ratio_w, ratio_h)
    new_shape_h = int(h * r)
    new_shape_w = int(w * r)
    
    if r < 1.:
        resize_img = cv2.resize(img, (new_shape_w, new_shape_h), interpolation=cv2.INTER_AREA)
    else:
        resize_img = cv2.resize(img, (new_shape_w, new_shape_h), interpolation=cv2.INTER_LINEAR)
    
    new_img_file = img_file.split('/')[-1]
    if train:
        cv2.imwrite('/usr/src/app/CrowdCounting-P2PNet/yolo_NWPU/images/train/' + new_img_file, resize_img)
    else:
        cv2.imwrite('/usr/src/app/CrowdCounting-P2PNet/yolo_NWPU/images/val/' + new_img_file, resize_img)
    
    txt_file = '/usr/src/app/CrowdCounting-P2PNet/NWPUCrowd/annotation_box_txt/' + os.path.splitext(new_img_file)[0] + '.txt'
    new_txt_file = os.path.splitext(new_img_file)[0] + '.txt'

    file = open(txt_file, 'r')
    txt_lines = file.read().split('\n')
    file.close()
    contents = ''

    for txt_line in txt_lines:
        if txt_line == '':
            break
        try:
            txt_line = txt_line.split(' ')
            x1 = float(txt_line[0])
            y1 = float(txt_line[1])
            x2 = float(txt_line[2])
            y2 = float(txt_line[3])
            x = (x1 + x2) / 2.
            y = (y1 + y2) / 2.
            width = x2 - x1
            height = y2 - y1
            x = x / w
            y = y / h
            width = width / w
            height = height / h
            contents +=  '0 ' + str(x) + ' ' + str(y) + ' ' + str(width) + ' ' + str(height) + '\n'
            
        except Exception as e:
            print(txt_line)

    if train:
        file = open('/usr/src/app/CrowdCounting-P2PNet/yolo_NWPU/labels/train/' + new_txt_file, 'w')
    else:
        file = open('/usr/src/app/CrowdCounting-P2PNet/yolo_NWPU/labels/val/' + new_txt_file, 'w')

    file.writelines(contents)
    file.close()

Invalid SOS parameters for sequential JPEG


In [16]:
import cv2
test_path = '/usr/src/app/CrowdCounting-P2PNet/yolo_NWPU/images/train/0019.jpg'
test_txt = '/usr/src/app/CrowdCounting-P2PNet/yolo_NWPU/labels/train/0019.txt'

a = cv2.imread(test_path)
f = open(test_txt, 'r')
b = f.read().split('\n')
height, width, channel = a.shape

for idx in range(len(b)):
    d = b[idx].split(' ')
    if len(d) < 3:
        break
    x, y, w, h = d[0], d[1], d[2], d[3]
    x = float(x)
    y = float(y)
    w = float(w)
    h = float(h)
    x1 = x - w / 2.
    y1 = y - h / 2.
    x2 = x + w / 2.
    y2 = y + h / 2.
    
    a = cv2.rectangle(a, (int(x1*width), int(y1*height)), (int(x2*width), int(y2*height)), (0, 0, 255), 2)

cv2.imwrite('./test.jpg', a)

True

In [15]:
b[0].split(' ')

['0.15235690235690236',
 '0.9653130287648054',
 '0.02499999999999999',
 '0.01598984771573599']

In [22]:
len(find_extlist('/usr/src/app/CrowdCounting-P2PNet/yolo_NWPU/images/val', '.jpg'))

721

In [24]:
len(find_extlist('/usr/src/app/CrowdCounting-P2PNet/yolo_NWPU/labels/val', '.txt'))

721

In [27]:
len(find_extlist('/usr/src/app/CrowdCounting-P2PNet/yolo_NWPU/images/train', '.jpg'))

2888

In [29]:
len(find_extlist('/usr/src/app/CrowdCounting-P2PNet/yolo_NWPU/labels/train', '.txt'))

2888

In [4]:
import numpy as np

z = np.zeros((4, 4))
n = np.arange(1, 5)
n = n.reshape(2, 2)

In [7]:
z[1:3, 1:3] = n

In [8]:
z

array([[0., 0., 0., 0.],
       [0., 1., 2., 0.],
       [0., 3., 4., 0.],
       [0., 0., 0., 0.]])